<a href="https://colab.research.google.com/github/wojciechnowaczyk/Statystyka/blob/main/Korelacja_i_regresja.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

Zadanie 1. Dla 15 gospodarstw domowych 4-osobowych zebrano informacje o przychodach miesięcznych (w zł/1 osobę) i wysokości wydatków na utrzymanie mieszkania (w zł/1osobę ) w I półroczu 2022 r.

Y – wydatki na utrzymanie mieszkania w przeliczeniu na 1 osobę.

X – przychody miesięczne w zł / 1 osobę.

Określ siłę i kierunek zależności między tymi zmiennymi.
Jeżeli zmienne są skorelowane, to
- znajdź funkcję regresji,
- zinterpretuj współczynnik regresji,
- sporządź wykres danych źródłowych i funkcji regresji,
- oceń jakość oszacowanej funkcji regresji,
- podaj prognozę wydatków na utrzymanie mieszkania, jeżeli przychody miesięczne na osobę wyniosą 2900 zł.

In [9]:
X = [2594, 2450, 2395, 2520, 2648, 2695, 2825, 2800, 2687, 2650, 2430, 2874, 2808, 2609, 2765]
Y = [580, 480, 420, 620, 700, 750, 950, 920, 660, 720, 450, 880, 850, 650, 740]
upkeep_cost_df = pd.DataFrame({'income': X, 'expense': Y})
upkeep_cost_df


,income,expense
0,2594,580
1,2450,480
2,2395,420
3,2520,620
4,2648,700
5,2695,750
6,2825,950
7,2800,920
8,2687,660
9,2650,720


In [12]:
# korelacja liniowa
income_avg = upkeep_cost_df['income'].mean()
expense_avg = upkeep_cost_df['expense'].mean()
#odchylenia
upkeep_cost_df['income_diff'] = upkeep_cost_df['income'] - income_avg
upkeep_cost_df['expense_diff'] = upkeep_cost_df['expense'] - expense_avg
upkeep_cost_df


,income,expense,income_diff,expense_diff
0,2594,580,-56.0,-111.333333
1,2450,480,-200.0,-211.333333
2,2395,420,-255.0,-271.333333
3,2520,620,-130.0,-71.333333
4,2648,700,-2.0,8.666667
5,2695,750,45.0,58.666667
6,2825,950,175.0,258.666667
7,2800,920,150.0,228.666667
8,2687,660,37.0,-31.333333
9,2650,720,0.0,28.666667


In [13]:
#korelacja
correlation = sum(upkeep_cost_df['income_diff'] * upkeep_cost_df['expense_diff']) / (
    sum(upkeep_cost_df['income_diff'] ** 2)*sum(upkeep_cost_df['expense_diff']**2)
)**0.5
correlation

0.9584246651529627

Zadanie 2. Pewne biuro obrotu nieruchomościami w Poznaniu oferowało na sprzedaż w dniu 1.12.2022 r. 25 mieszkań. Dane odnośnie ich powierzchni i ceny przedstawiono w ramce offer_df.
Określ siłę i kierunek zależności między tymi zmiennymi. Jeżeli zmienne są skorelowane, to
- znajdź funkcję regresji, zinterpretuj współczynnik regresji, sporządź wykres danych źródłowych i funkcji regresji, oceń jakość oszacowanej funkcji regresji,
- podaj prognozę ceny za m2 w przypadku, gdy mieszkanie będzie miało 85 m2.

In [ ]:
surface = [80, 67, 87, 87, 85, 36, 46, 63, 95, 140, 38, 30, 30, 61, 47, 30, 38, 50, 81, 33, 38, 38, 30, 38, 38] # w m2
price = [825, 710, 910, 945, 920, 480, 510, 605, 1100, 1390, 420, 410, 290, 620, 545, 320, 450, 590, 890, 420, 420, 450, 420, 410, 430] # w tys. zł
offer_df = pd.DataFrame({'surface': surface, 'price': price})
offer_df
